In [2]:
import pandas as pd
import numpy as np

In [36]:
df1 = pd.read_csv("Noah_5Years_002.csv", skiprows=4)
df2 = pd.read_csv("Noah_5Years_003.csv", skiprows=4)

df = pd.concat([df1, df2])
df = df[['Timestamp', 'Noah - Water Use (gal)']]
df[['Date', 'Time']] = df['Timestamp'].str.split(' ', expand = True)
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day_name()
df = df[['Year', 'Month', 'Day', 'Time', 'Noah - Water Use (gal)']]

year_bool_array = df['Year'] >= 2021
month_bool_array = ((df['Month'] >= 9) & (df['Month'] <= 12)) | ((df['Month'] >= 2) & (df['Month'] <= 5))

df = df[year_bool_array & month_bool_array]
df = df[['Day', 'Time', 'Noah - Water Use (gal)']]
df.columns = ['Day', 'Time', 'Water']
df.dropna(inplace=True)
df


,Day,Time,Water
8121,Monday,00:00,3.49389
8122,Monday,00:15,42.85861
8123,Monday,00:30,9.82386
8124,Monday,00:45,2.69802
8125,Monday,01:00,2.83658
...,...,...,...
25069,Saturday,22:45,0.86475
25070,Saturday,23:00,3.35533
25071,Saturday,23:15,2.83638
25072,Saturday,23:30,0.89934


In [37]:
df.isnull().any()

Day      False
Time     False
Water    False
dtype: bool

In [38]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

X = df.drop('Water', axis=1)
y = df['Water']

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train_transformed = OneHotEncoder().fit_transform(X_train)
X_test_transformed = OneHotEncoder().fit_transform(X_test)


In [40]:
rgs = RandomForestRegressor()
rgs.fit(X_train_transformed, y_train)
y_pred = rgs.predict(X_test_transformed)

In [45]:
pd.Series(y_pred)

0         3.689308
1        13.312761
2        22.020838
3        19.973985
4        21.645586
           ...    
11565    15.992107
11566    26.178880
11567    21.221427
11568    19.200320
11569    23.416485
Length: 11570, dtype: float64

In [44]:
y_test

16466     3.00933
46972    29.71381
35159    17.64220
32328    33.13832
280      35.24821
           ...   
2576     22.13790
315      61.01796
220      14.42493
17602    54.44756
21667    17.39937
Name: Water, Length: 11570, dtype: float64

In [46]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.09303030660052303